In [ ]:
from torch import nn
import torch
import torchvision
from torchvision import transforms
import math
import time
from ptflops import get_model_complexity_info

In [ ]:
class AlexNet(nn.Module):
    def __init__(self, lr=0.1, num_classes=10):
        super().__init__()
        self.net = nn.Sequential(
            nn.LazyConv2d(96, kernel_size=11, stride=4, padding=1),
            nn.ReLU(), nn.MaxPool2d(kernel_size=3, stride=2),
            nn.LazyConv2d(256, kernel_size=5, padding=2), nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.LazyConv2d(384, kernel_size=3, padding=1), nn.ReLU(),
            nn.LazyConv2d(384, kernel_size=3, padding=1), nn.ReLU(),
            nn.LazyConv2d(256, kernel_size=3, padding=1), nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2), nn.Flatten(),
            nn.LazyLinear(4096), nn.ReLU(), nn.Dropout(p=0.5),
            nn.LazyLinear(4096), nn.ReLU(),nn.Dropout(p=0.5),
            nn.LazyLinear(num_classes))
    def forward(self, X):
        return self.net(X)

In [ ]:
device = torch.device('cpu')

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize((224,224)),
     transforms.Normalize((0.5,), (0.5,))])

# Create datasets for training & validation, download if necessary
training_set = torchvision.datasets.FashionMNIST('./data', train=True, transform=transform, download=True)
validation_set = torchvision.datasets.FashionMNIST('./data', train=False, transform=transform, download=True)

# Create data loaders for our datasets; shuffle for training, not for validation
training_loader = torch.utils.data.DataLoader(training_set, batch_size=128, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=128, shuffle=False)

In [ ]:
model = AlexNet()
model.to(device)

In [ ]:
device

In [ ]:
# import torchvision.models as models
# import torch
# from ptflops import get_model_complexity_info

# with torch.device("mps"):
#     macs, params = get_model_complexity_info(model, (1, 224, 224), as_strings=True,
#                                            print_per_layer_stat=True, verbose=True)
#     print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
#     print('{:<30}  {:<8}'.format('Number of parameters: ', params))


In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)
for i in range(100):
    start_time = time.time()
    cnt = 0
    for x, y in training_loader:
        cnt += 1
        x = x.to(device)
        y = y.to(device)
        # model.train()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        optimizer.zero_grad()
        with torch.no_grad():
            loss.backward()
            optimizer.step()

        end_time = time.time()
        print(f"Epoche {i}, Batch {cnt} ({end_time - start_time} s): Loss = {loss.item()}", end = "\r")

    # model.eval()
    y_pred = model(x)
    loss = criterion(y_pred, y)
    print(f"Epoche {i} ({end_time - start_time} s): Loss = {loss.item()}")

In [ ]:
class Model2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.W1 = torch.nn.Parameter(torch.randn((3, 10)))
        self.b1 = torch.nn.Parameter(torch.randn((10)))
        self.W2 = torch.nn.Parameter(torch.randn((10, 1)))
        self.b2 = torch.nn.Parameter(torch.randn((1)))

    def forward(self, X):
        X = torch.mm(X, self.W1) + self.b1
        X = torch.max(X, torch.zeros_like(X))
        X = torch.mm(X, self.W2) + self.b2
        return X

In [ ]:
model2 = Model2()
criterion = torch.nn.MSELoss(reduction="mean")
optimizer = torch.optim.SGD(model2.parameters(), lr = 0.0001)
for i in range(10000):
    for x, y in iter(dataloader):
        # model.train()
        y_pred = model2(x)
        loss = criterion(y_pred, y)
        if (i % 10 == 9):
            print(f"{i}: {loss.item()}", end = "\r")
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if (i % 10 == 9):
        print()

In [ ]:
for param in model.parameters():
    print(param.size())

In [ ]:
for param in model2.parameters():
    print(param.size())

In [ ]:
a = torch.zeros((5,2))
b = torch.ones((5))
*a

In [ ]:
a = tuple([1,2,3])
*a

In [ ]:
print(type(model(x)))